## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [2]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [4]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000250E99278C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000250E993C410>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Hello Krish, it's nice to meet you!\n\nBeing a Chief AI Engineer is a fascinating role. What kind of projects are you working on these days? Are there any particular challenges or advancements in the field that you're particularly excited about?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 22, 'total_tokens': 76, 'completion_time': 0.098181818, 'prompt_time': 0.000139049, 'queue_time': 0.014645871, 'total_time': 0.098320867}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bb52ba50-0dd7-4427-81ba-6f61a0860866-0', usage_metadata={'input_tokens': 22, 'output_tokens': 54, 'total_tokens': 76})

In [6]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You said your name is Krish and that you are a Chief AI Engineer. \n\nIs there anything else you'd like to tell me about yourself or your work? I'm happy to chat! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 97, 'total_tokens': 144, 'completion_time': 0.085454545, 'prompt_time': 0.002500511, 'queue_time': 0.010966258, 'total_time': 0.087955056}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8cc3b550-abc6-465c-b5fb-9fa58473c35a-0', usage_metadata={'input_tokens': 97, 'output_tokens': 47, 'total_tokens': 144})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
!pip install langchain_community

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [8]:
config={"configurable":{"session_id":"chat1"}}

In [22]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"), 
     HumanMessage(content="Prakash is my student")],
    config=config
)

In [23]:
response.content

"That's great, Krish!  \n\nIt sounds like you have an interesting and important role as a Chief AI Engineer.  Is there anything you'd like to discuss about AI or your work? Perhaps you have questions about a specific project, or you're looking for information on a particular AI concept?\n\nI'm happy to chat about anything related to AI, or even just general topics if you prefer.  \n\nLet me know how I can be helpful! 😊\n"

In [ ]:
with_message_history.invoke(
    [HumanMessage(content="my students?")],
    config=config,
)

AIMessage(content="You mentioned Prakash is your student.  Do you have any other students you'd like to talk about? \n\nOr are you curious about something specific regarding Prakash?   I'm ready to listen! 😄  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 261, 'total_tokens': 309, 'completion_time': 0.087272727, 'prompt_time': 0.010233835, 'queue_time': 0.003487555, 'total_time': 0.097506562}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c82f3d97-3e38-4928-81a0-69419375b575-0', usage_metadata={'input_tokens': 261, 'output_tokens': 48, 'total_tokens': 309})

In [12]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name.\n\nIf you'd like to tell me your name, I'd be happy to use it! 😊\n"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you. 👋 \n\nWhat can I do for you today?\n"

In [14]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"Your name is John, remember? 😊  I'm happy to help with anything else you need! \n"

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [16]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]})

AIMessage(content="Hello Krish, it's nice to meet you!\n\nI'm happy to help with any questions you have.  What can I do for you today? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 31, 'total_tokens': 69, 'completion_time': 0.069090909, 'prompt_time': 0.000306849, 'queue_time': 0.014156141, 'total_time': 0.069397758}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-91c3dc77-cc6b-474f-aa46-9ab4eb356651-0', usage_metadata={'input_tokens': 31, 'output_tokens': 38, 'total_tokens': 69})

In [17]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [18]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Krish")],
    config=config
)

response

AIMessage(content="Hello Krish, it's nice to meet you!\n\nI'm happy to help answer any questions you have to the best of my ability.  What can I do for you today? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 31, 'total_tokens': 75, 'completion_time': 0.08, 'prompt_time': 0.000372389, 'queue_time': 0.012672751, 'total_time': 0.080372389}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7b6756eb-2753-45db-b677-ff146cb667ba-0', usage_metadata={'input_tokens': 31, 'output_tokens': 44, 'total_tokens': 75})

In [19]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Krish, you told me! 😄  \n\nIs there anything else I can help you with?\n'

In [27]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [21]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")],"language":"Hindi"})
response.content

'नमस्ते कृष्ण! \n\nमुझे बहुत खुशी है आपसे मिलने में। आप मुझसे कुछ भी पूछ सकते हैं, मैं अपनी पूरी कोशिश करूँगा आपके सवाल का उत्तर देना। \n\nआपका दिन कैसा चल रहा है?\n\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [25]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [28]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते कृष्ण! 😊  मुझे बहुत खुशी है कि आप मुझसे जुड़ रहे हैं। \n\nआप मुझे क्या पूछना चाहेंगे? मैं अपनी पूरी क्षमता के साथ उत्तर देने की कोशिश करूँगा।  \n'

In [29]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [31]:
response.content

'आपका नाम कृष्ण है! 😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [32]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

c:\Users\praka\VSCodeProjects\python\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\praka\VSCodeProjects\python\venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\praka\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer 

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [33]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")], # because of trimmer the info is lost.
    "language":"English"
    }
)
response.content

"As a helpful assistant, I don't have access to your personal information, including your ice cream preferences.\n\nWhat's your favorite flavor?  🍦😊\n"

In [34]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "What is 2 + 2?".  \n\n\n\nIs there anything else I can help you with?\n'

In [35]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [36]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name.\n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n\n"

In [37]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n\n"